# VII. Timing Code

Julia has a built-in macro `@time` that can be used to time code.

In [1]:
using LinearAlgebra
using Random

Random.seed!(1234);

In [2]:
# Exponentiate a matrix

function ExpMat(A,n)
    nr = size(A,1)
    I = Diagonal(ones(nr,nr))
    s = I + A
    t = A
    for j = 2:n
        s += t*A / n
    end
    return s
end

ExpMat (generic function with 1 method)

In [3]:
U = randn(1000,1000);

The first time Julia encounters a user-written function it will compile and then cache this compiled function. Therefore the first very first timing will include the compilation time. To get an accurate timing of the function's execution time you will want to time the function a few times. 

In [4]:
@time ExpMat(U, 20);

  2.561447 seconds (3.93 M allocations: 635.974 MiB, 10.67% gc time)


In [5]:
@time ExpMat(U, 20);

  1.270768 seconds (126 allocations: 450.147 MiB, 1.77% gc time)


In [6]:
@time ExpMat(U, 20);

  1.278146 seconds (126 allocations: 450.147 MiB, 1.85% gc time)


An alternative is to use the __BenchmarkTools__ package which will provide a benchmark based on a sample of runs and provides sample statistics.

In [7]:
using BenchmarkTools

In [8]:
@benchmark ExpMat(U, 20) seconds = 6

BenchmarkTools.Trial: 
  memory estimate:  450.15 MiB
  allocs estimate:  122
  --------------
  minimum time:     1.243 s (1.40% GC)
  median time:      1.280 s (1.68% GC)
  mean time:        1.308 s (4.16% GC)
  maximum time:     1.440 s (12.79% GC)
  --------------
  samples:          5
  evals/sample:     1

We saw before there were three ways to do apply a function to a collection of values: array comprehension, map function, using the dot syntax, and a for loop. We can benchmark these approaches to see how they compare.

In [9]:
array = randn(10^6);

In [10]:
@benchmark [exp(cos(u)) for u in array] samples = 20

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  3
  --------------
  minimum time:     23.284 ms (0.00% GC)
  median time:      23.576 ms (0.00% GC)
  mean time:        24.743 ms (1.17% GC)
  maximum time:     28.660 ms (4.60% GC)
  --------------
  samples:          20
  evals/sample:     1

In [11]:
@benchmark map(x -> exp(cos(x)), array) samples = 20

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  3
  --------------
  minimum time:     23.233 ms (0.00% GC)
  median time:      23.376 ms (0.00% GC)
  mean time:        24.599 ms (1.28% GC)
  maximum time:     28.503 ms (4.35% GC)
  --------------
  samples:          20
  evals/sample:     1

In [12]:
@benchmark exp.(cos.(array)) samples = 20

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  7
  --------------
  minimum time:     22.677 ms (0.00% GC)
  median time:      22.953 ms (0.00% GC)
  mean time:        24.045 ms (1.27% GC)
  maximum time:     28.443 ms (3.95% GC)
  --------------
  samples:          20
  evals/sample:     1

In [13]:
function floop(array)
    for i in 1:length(array); 
        exp(cos(array[i])); 
    end
    nothing
end

floop (generic function with 1 method)

In [14]:
@benchmark floop(array) samples = 20

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     22.475 ms (0.00% GC)
  median time:      22.644 ms (0.00% GC)
  mean time:        22.697 ms (0.00% GC)
  maximum time:     23.370 ms (0.00% GC)
  --------------
  samples:          20
  evals/sample:     1